In [8]:
import pandas as pd
import numpy as np

# --- 1. TẠO DỮ LIỆU GIẢ LẬP (MOCK DATA) ---
# Giả định dữ liệu bệnh viện gồm các cột cần thiết:
# 'Mã BN': ID bệnh nhân
# 'Khoa': Khoa khám bệnh (Nội, Ngoại, Nhi,...)
# 'Chi phí khám': Tổng chi phí dịch vụ
# 'Tỷ lệ BHYT': Phần trăm chi phí được BHYT chi trả (từ 0 đến 1)
# 'Tuổi': Tuổi của bệnh nhân
# 'Ngày khám': Giả lập cho việc đếm số lượt khám

data = {
    'Mã BN': [101, 102, 103, 101, 104, 105, 106, 103, 107, 108, 109, 110, 102, 101, 111],
    'Khoa': ['Nội', 'Ngoại', 'Nhi', 'Nội', 'Ngoại', 'Tim Mạch', 'Nhi', 'Nhi', 'Nội', 'Ngoại', 'Tim Mạch', 'Nội', 'Ngoại', 'Nội', 'Tim Mạch'],
    'Chi phí khám': [500000, 850000, 300000, 550000, 900000, 1200000, 350000, 320000, 600000, 800000, 1300000, 520000, 880000, 480000, 1250000],
    'Tỷ lệ BHYT': [0.5, 0.8, 0.2, 0.5, 0.8, 0.9, 0.2, 0.2, 0.5, 0.8, 0.9, 0.5, 0.8, 0.5, 0.9],
    'Tuổi': [45, 30, 5, 45, 55, 65, 8, 5, 70, 40, 62, 50, 30, 45, 68]
}

df = pd.DataFrame(data)
# Thêm cột 'Tổng chi phí thực' để phục vụ việc phân tích nâng cao (tổng chi phí khám cho từng bệnh nhân)
df['Tổng chi phí thực'] = df['Chi phí khám'] * (1 - df['Tỷ lệ BHYT'])

print("--- Dữ liệu gốc (5 dòng đầu) ---")
print(df.head())
print("\n" + "="*50 + "\n")


# --- 2. THỰC HIỆN YÊU CẦU DÙNG GROUPBY VÀ AGG ---

# Yêu cầu 1: Tính chi phí trung bình theo khoa.
print("## 1. Chi phí trung bình theo khoa")
df_chi_phi_khoa = df.groupby('Khoa').agg(
    Chi_phi_Trung_binh = ('Chi phí khám', 'mean')
).reset_index()
print(df_chi_phi_khoa)
print("\n" + "="*50 + "\n")

# Yêu cầu 2: Tính số lượt khám theo bệnh nhân (Sử dụng 'count' cho bất kỳ cột nào, ví dụ 'Khoa')
print("## 2. Số lượt khám theo bệnh nhân")
df_luot_kham_bn = df.groupby('Mã BN').agg(
    So_luot_kham = ('Khoa', 'count')
).reset_index()
print(df_luot_kham_bn)
print("\n" + "="*50 + "\n")


# Yêu cầu 3: Tính tỷ lệ BHYT trung bình (cho toàn bộ dữ liệu)
# Lưu ý: Yêu cầu này không cần GroupBy, nhưng có thể tính Trung bình BHYT theo Khoa để phân tích sâu hơn.
print("## 3. Tỷ lệ BHYT trung bình (Toàn cục)")
ty_le_bhyt_tb_toan_cuc = df['Tỷ lệ BHYT'].mean()
print(f"Tỷ lệ BHYT trung bình toàn cục: {ty_le_bhyt_tb_toan_cuc:.4f}")

# (Mở rộng) Tính tỷ lệ BHYT trung bình theo khoa
print("\n## 3. Mở rộng: Tỷ lệ BHYT trung bình theo khoa")
df_bhyt_khoa = df.groupby('Khoa').agg(
    Ty_le_BHYT_TB = ('Tỷ lệ BHYT', 'mean')
).reset_index()
print(df_bhyt_khoa)
print("\n" + "="*50 + "\n")


# Yêu cầu 4: Xác định khoa có chi phí cao nhất.
print("## 4. Khoa có chi phí khám trung bình cao nhất")
# Dựa trên kết quả từ Yêu cầu 1
khoa_chi_phi_max = df_chi_phi_khoa.loc[df_chi_phi_khoa['Chi_phi_Trung_binh'].idxmax()]
print(f"Khoa có chi phí trung bình cao nhất: {khoa_chi_phi_max['Khoa']}")
print(f"Chi phí trung bình: {khoa_chi_phi_max['Chi_phi_Trung_binh']:.0f} VND")
print("\n" + "="*50 + "\n")


# Yêu cầu 5: Phân tích liên hệ giữa tuổi và chi phí khám.
print("## 5. Phân tích liên hệ giữa tuổi và chi phí khám")
# Phương pháp 1: Tính hệ số tương quan (Correlation Coefficient)
correlation = df['Tuổi'].corr(df['Chi phí khám'])
print(f"Hệ số tương quan Pearson (Tuổi vs Chi phí khám): {correlation:.4f}")

# Phương pháp 2: Grouping theo nhóm tuổi (binning)
bins = [0, 18, 40, 60, 100]
labels = ['Thiếu nhi', 'Người trẻ (19-40)', 'Trung niên (41-60)', 'Người cao tuổi (61+)']
df['Nhom Tuoi'] = pd.cut(df['Tuổi'], bins=bins, labels=labels, right=True)

df_chi_phi_theo_tuoi = df.groupby('Nhom Tuoi', observed=True).agg(
    Chi_phi_Trung_binh = ('Chi phí khám', 'mean'),
    So_luot_kham = ('Khoa', 'count')
).reset_index()

print("\nChi phí trung bình theo nhóm tuổi:")
print(df_chi_phi_theo_tuoi)
print("\n" + "="*50 + "\n")

--- Dữ liệu gốc (5 dòng đầu) ---
   Mã BN   Khoa  Chi phí khám  Tỷ lệ BHYT  Tuổi  Tổng chi phí thực
0    101    Nội        500000         0.5    45           250000.0
1    102  Ngoại        850000         0.8    30           170000.0
2    103    Nhi        300000         0.2     5           240000.0
3    101    Nội        550000         0.5    45           275000.0
4    104  Ngoại        900000         0.8    55           180000.0


## 1. Chi phí trung bình theo khoa
       Khoa  Chi_phi_Trung_binh
0     Ngoại        8.575000e+05
1       Nhi        3.233333e+05
2       Nội        5.300000e+05
3  Tim Mạch        1.250000e+06


## 2. Số lượt khám theo bệnh nhân
    Mã BN  So_luot_kham
0     101             3
1     102             2
2     103             2
3     104             1
4     105             1
5     106             1
6     107             1
7     108             1
8     109             1
9     110             1
10    111             1


## 3. Tỷ lệ BHYT trung bình (Toàn cục)
Tỷ 

In [6]:
print("\n\n--- 4. Phân tích Ý nghĩa của Bảng Tổng Hợp ---")
print("\n**4.1. Phân tích theo Giới tính (Gender):**")
print("* **Tỷ lệ:** Cho biết sự chênh lệch về số lượng bệnh nhân giữa Nam và Nữ trong tập dữ liệu.")
print(f"* **Ví dụ:** Nếu 'F' có tổng số bệnh nhân cao hơn (như trong bảng trên), có thể gợi ý rằng tập dữ liệu này có xu hướng nghiêng về bệnh nhân nữ, hoặc một loại bệnh cụ thể đang được nghiên cứu có tỷ lệ mắc cao hơn ở nữ giới.")
print(f"* **Tuổi:** So sánh 'Tuổi Trung Bình' giúp xác định liệu có sự khác biệt đáng kể về độ tuổi khám/chữa bệnh giữa hai giới tính.")

print("\n**4.2. Phân tích theo Họ và tên (Full Name):**")
print("* **Tần suất:** 'Tổng Số Lần Xuất Hiện' cho thấy tên nào là phổ biến nhất trong tập dữ liệu. Điều này có thể hữu ích trong việc làm sạch dữ liệu (ví dụ: kiểm tra trùng lặp) hoặc phân tích các nhóm có tên giống nhau.")
print("* **Độ Lệch Chuẩn (Std):** Đây là chỉ số quan trọng.")
print("    * **Std lớn:** Cho thấy những người cùng tên có sự phân tán rộng về độ tuổi (ví dụ: Le Van C có thể bao gồm một người 25 tuổi và một người 50 tuổi).")
print("    * **Std nhỏ/bằng 0:** Cho thấy những người cùng tên có độ tuổi gần như nhau hoặc chính xác là một độ tuổi (nếu chỉ có 1 lần xuất hiện, Std=0).")



--- 4. Phân tích Ý nghĩa của Bảng Tổng Hợp ---

**4.1. Phân tích theo Giới tính (Gender):**
* **Tỷ lệ:** Cho biết sự chênh lệch về số lượng bệnh nhân giữa Nam và Nữ trong tập dữ liệu.
* **Ví dụ:** Nếu 'F' có tổng số bệnh nhân cao hơn (như trong bảng trên), có thể gợi ý rằng tập dữ liệu này có xu hướng nghiêng về bệnh nhân nữ, hoặc một loại bệnh cụ thể đang được nghiên cứu có tỷ lệ mắc cao hơn ở nữ giới.
* **Tuổi:** So sánh 'Tuổi Trung Bình' giúp xác định liệu có sự khác biệt đáng kể về độ tuổi khám/chữa bệnh giữa hai giới tính.

**4.2. Phân tích theo Họ và tên (Full Name):**
* **Tần suất:** 'Tổng Số Lần Xuất Hiện' cho thấy tên nào là phổ biến nhất trong tập dữ liệu. Điều này có thể hữu ích trong việc làm sạch dữ liệu (ví dụ: kiểm tra trùng lặp) hoặc phân tích các nhóm có tên giống nhau.
* **Độ Lệch Chuẩn (Std):** Đây là chỉ số quan trọng.
    * **Std lớn:** Cho thấy những người cùng tên có sự phân tán rộng về độ tuổi (ví dụ: Le Van C có thể bao gồm một người 25 tuổi và một người 50 t

In [9]:
# --- 1.5. BỔ SUNG CÁC CỘT TÍNH TOÁN VỀ CHI PHÍ ---
# Kiểm tra tồn tại của biến df và các cột cần thiết trước khi tính toán
required_for_cost = ['Chi phí khám', 'Tỷ lệ BHYT']
if 'df' not in globals() and 'df' not in locals():
    raise NameError('Biến `df` chưa được định nghĩa. Hãy chạy cell tải dữ liệu trước.')
missing_cost_cols = [c for c in required_for_cost if c not in df.columns]
if missing_cost_cols:
    raise KeyError(f'Không tìm thấy các cột cần thiết để tính chi phí: {missing_cost_cols}')
# Đảm bảo kiểu số cho các cột liên quan
df['Chi phí khám'] = pd.to_numeric(df['Chi phí khám'], errors='coerce').fillna(0.0)
df['Tỷ lệ BHYT'] = pd.to_numeric(df['Tỷ lệ BHYT'], errors='coerce').fillna(0.0)
# Tính 1: Chi phí BHYT chi trả (Số tiền)
df['Chi phí BHYT'] = df['Chi phí khám'] * df['Tỷ lệ BHYT']
# Tính 2: Chi phí thực tế bệnh nhân phải trả
# Chi phí Bệnh nhân = Chi phí khám - Chi phí BHYT
df['Chi phí Bệnh nhân'] = df['Chi phí khám'] - df['Chi phí BHYT']
print("--- Dữ liệu sau khi tính Chi phí BHYT và Chi phí Bệnh nhân (5 dòng đầu) ---")
print(df.head())
print("\n" + "="*50 + "\n")

print("## 6. Tổng và Trung bình Chi phí BHYT chi trả theo Khoa")
df_bhyt_tong_hop = df.groupby('Khoa').agg(
    Tong_BHYT_chi_tra = ('Chi phí BHYT', 'sum'),
    TB_BHYT_chi_tra = ('Chi phí BHYT', 'mean')
).reset_index()
print(df_bhyt_tong_hop)
print("\n" + "="*50 + "\n")

print("## 7. Tổng và Trung bình Chi phí Bệnh nhân phải trả theo Khoa")
df_bn_tong_hop = df.groupby('Khoa').agg(
    Tong_BN_thanh_toan = ('Chi phí Bệnh nhân', 'sum'),
    TB_BN_thanh_toan = ('Chi phí Bệnh nhân', 'mean')
).reset_index()
print(df_bn_tong_hop)
print("\n" + "="*50 + "\n")

print("## 8. Phân tích Chi phí theo Tần suất khám của Bệnh nhân")

# 1. Tính tổng chi phí và tổng lượt khám cho MỖI BỆNH NHÂN
df_patient_summary = df.groupby('Mã BN').agg(
    Tong_luot_kham = ('Khoa', 'count'),
    Tong_chi_phi_BN = ('Chi phí Bệnh nhân', 'sum'),
    Tong_chi_phi_BHYT = ('Chi phí BHYT', 'sum')
).reset_index()

# 2. Tạo nhóm Tần suất khám (Ví dụ: Nhóm 1, Nhóm 2+, Nhóm 3+)
bins_freq = [0, 1, 2, 100] # Giả định tối đa 100 lượt khám
labels_freq = ['Kham 1 lần', 'Kham 2 lần', 'Kham 3+ lần']
df_patient_summary['Nhom Tan Suat'] = pd.cut(df_patient_summary['Tong_luot_kham'], bins=bins_freq, labels=labels_freq, right=True)

# 3. Tính trung bình chi phí cho từng nhóm tần suất
df_freq_analysis = df_patient_summary.groupby('Nhom Tan Suat', observed=True).agg(
    So_luong_BN = ('Mã BN', 'count'),
    TB_Tong_Chi_phi_BN = ('Tong_chi_phi_BN', 'mean'),
    TB_Tong_Chi_phi_BHYT = ('Tong_chi_phi_BHYT', 'mean')
).reset_index()

print("\nKết quả phân tích chi phí trung bình theo nhóm Tần suất khám:")
print(df_freq_analysis)
print("\n" + "="*50 + "\n")

print("## 9. Khoa có Tỷ lệ Chi phí Bệnh nhân/Tổng chi phí cao nhất")

# 1. Tính tổng chi phí cho mỗi khoa
df_khoa_tong_chi_phi = df.groupby('Khoa').agg(
    Tong_Chi_phi_Kham = ('Chi phí khám', 'sum'),
    Tong_Chi_phi_BN = ('Chi phí Bệnh nhân', 'sum')
).reset_index()

# 2. Tính tỷ lệ Chi phí Bệnh nhân trên Tổng Chi phí
df_khoa_tong_chi_phi['Ty_le_BN_Tren_Tong'] = (
    df_khoa_tong_chi_phi['Tong_Chi_phi_BN'] / df_khoa_tong_chi_phi['Tong_Chi_phi_Kham']
)

# 3. Sắp xếp để tìm khoa có tỷ lệ cao nhất
df_khoa_ty_le_max = df_khoa_tong_chi_phi.sort_values(
    by='Ty_le_BN_Tren_Tong', ascending=False
)

print("\nCác khoa theo Tỷ lệ Chi phí Bệnh nhân phải trả trên Tổng Chi phí:")
print(df_khoa_ty_le_max[['Khoa', 'Tong_Chi_phi_Kham', 'Tong_Chi_phi_BN', 'Ty_le_BN_Tren_Tong']])
print("\n" + "="*50 + "\n")

--- Dữ liệu sau khi tính Chi phí BHYT và Chi phí Bệnh nhân (5 dòng đầu) ---
   Mã BN   Khoa  Chi phí khám  Tỷ lệ BHYT  Tuổi  Tổng chi phí thực  \
0    101    Nội        500000         0.5    45           250000.0   
1    102  Ngoại        850000         0.8    30           170000.0   
2    103    Nhi        300000         0.2     5           240000.0   
3    101    Nội        550000         0.5    45           275000.0   
4    104  Ngoại        900000         0.8    55           180000.0   

            Nhom Tuoi  Chi phí BHYT  Chi phí Bệnh nhân  
0  Trung niên (41-60)      250000.0           250000.0  
1   Người trẻ (19-40)      680000.0           170000.0  
2           Thiếu nhi       60000.0           240000.0  
3  Trung niên (41-60)      275000.0           275000.0  
4  Trung niên (41-60)      720000.0           180000.0  


## 6. Tổng và Trung bình Chi phí BHYT chi trả theo Khoa
       Khoa  Tong_BHYT_chi_tra  TB_BHYT_chi_tra
0     Ngoại          2744000.0     6.860000e+05
1      

thêm để sửa mess